In [5]:
import os

neo4j_config = {
    "password": "password"
}

In [6]:
from copy import deepcopy
import json
import pprint
pp = pprint.PrettyPrinter(depth=6)

from py2neo import Graph
neo4j_con = Graph(**neo4j_config)

In [34]:
def get_recommendations(product_id):
    query_incoming_prods = """MATCH (p1:Product)-[r:BOUGHT_WITH]->(p2:Product) WHERE p2.product_id={product_id} 
                                AND p1.group=p2.group RETURN p1 ORDER BY p1.avg_rating DESC, toInteger(p1.salesrank)
                                LIMIT 50"""
    
    query_outgoing_prods = """MATCH (p1:Product)-[r:BOUGHT_WITH]->(p2:Product) WHERE p1.product_id={product_id} 
                                AND p1.group=p2.group RETURN p2 ORDER BY p2.avg_rating DESC, toInteger(p1.salesrank) 
                                LIMIT 50"""
    
    # Fetching incoming nodes TO p1. e.g. (p5, p6, p7) -> p1
    # Belonging to same group(category) and sorted on the basis of `avg_rating` & 'salesrank'
    cur = neo4j_con.run(query_incoming_prods, {"product_id": product_id})
    incoming_prods_nodes = []
    for node_tups in cur:
        for node_tup in node_tups.values():
            nodes_dict = {k: v for k, v in node_tup.iteritems()}
            incoming_prods_nodes.append(nodes_dict)
    
    # Fetching outgoing nodes FROM p1. e.g. p1 -> (p2, p3)
    # Belonging to same group(category) and sorted on the basis of `avg_rating` & 'salesrank'
    cur = neo4j_con.run(query_outgoing_prods, {"product_id": product_id})
    outgoing_prods_nodes = []
    for node_tups in cur:
        for node_tup in node_tups.values():
            nodes_dict = {k: v for k, v in node_tup.iteritems()}
            outgoing_prods_nodes.append(nodes_dict)
    
    return incoming_prods_nodes, outgoing_prods_nodes

In [35]:
product_id = 70

# Recommendations for product(product_id = 70)
incoming_prods_nodes, outgoing_prods_nodes = get_recommendations(product_id)

# Merging incoming & outgoing nodes. e.g. (p5, p6, p7) + (p2, p3)
incoming_prods_nodes.extend(outgoing_prods_nodes)
# print(incoming_prods_nodes)

# Sorting the nodes on the basis of `avg_rating`.
incoming_prods_nodes.sort(key=lambda prod: (-prod['avg_rating'], int(prod['salesrank'])))
recommended_prod_ids = set(map(lambda prod: prod["product_id"], incoming_prods_nodes))

print("For product with product_id = {} Recommended products are - \n{}".format(product_id, recommended_prod_ids))
# pp.pprint(incoming_prods_nodes)

For product with product_id = 70 Recommended products are - 
set([15, 421, 102, 103])
